In [ ]:
import sys
import time
import numpy as np
import pandas as pd
import itertools
import plotly.express as px
import plotly.graph_objects as go
import datetime

from sklearn.covariance import GraphicalLasso as sk_GL
from sklearn.covariance import empirical_covariance
from sklearn import set_config
set_config(print_changed_only=False)

from gglasso.solver.single_admm_solver import ADMM_SGL
from gglasso.solver.single_admm_solver import block_SGL
from gglasso.helper.data_generation import time_varying_power_network, group_power_network, sample_covariance_matrix
from gglasso.helper.model_selection import single_grid_search
from benchmarks import models_to_dict, sklearn_time_benchmark, admm_time_benchmark, model_solution, benchmark_parameters, plot_log_distance
from benchmarks import network_generation, hamming_distance, sparsity_benchmark, time_benchmark, dict_to_dataframe, dict_shape

from regain.covariance import GraphicalLasso as rg_GL

In [ ]:
S_dict=dict()
X_dict=dict()
Theta_dict=dict()

p_list=[100, 300, 1000]
N_list=[300, 600, 2000]

for p, N in zip(p_list, N_list):
    S, X, Theta = network_generation(p, N, K=1, M=2)
    
    S_dict[p, N] = S
    X_dict[p, N] = X
    Theta_dict[p, N] = Theta

print("\n Shape of S_i:", dict_shape(S_dict))
print("\n Shape of X_i:", dict_shape(X_dict))
print("\n Shape of Theta_i:", dict_shape(Theta_dict))

In [ ]:
sk_params, rg_params, admm_params = benchmark_parameters()

In [ ]:
sk_tol_list = [0.8, 0.5, 0.25, 0.1]
enet_list = [0.5, 0.25, 0.1]
X_1 = list(X_dict.values())[0]
sk_params = {"tol": sk_tol_list, "enet": enet_list}
models = models_to_dict(models=["sklearn"], lambda1=0.1, max_iter=50000, sk_params=sk_params)
models

In [ ]:
Z = sk_GL(alpha=0.1, max_iter=50000, tol=1e-8, enet_tol=0.01, assume_centered=True).fit(X_1)
Z_prec = Z.precision_

In [ ]:
t, ac, Z_sk = sklearn_time_benchmark(models=models, X=X_1, Z=Z_prec)

In [ ]:
sparsity = sparsity_benchmark(Theta_dict=Theta_dict, Z_dict=Z_sk, t_rounding=1e-4)

In [ ]:
df = pd.DataFrame(data={'name': t.keys(), 'time': t.values(), "accuracy": ac.values(), "hamming": sparsity.values()})
df['split'] = df['name'].str.split('_')
columns_names = ["method", "tol_str", "tol", "rtol_str", "rtol"]
df[columns_names] = pd.DataFrame(df['split'].tolist(), index=df['split'].index)

redundant_cols = ['split', "tol_str", "rtol_str"]
df = df.drop(redundant_cols, axis=1)

In [ ]:
df

In [ ]:
fig = px.scatter(df, x="time", y="accuracy", text = "tol", color= "method", 
              log_y = True,
              labels={
                     "time": "Time, s",
                     "accuracy": "Log_distance",
                     "method": "method"
                 },
              template = "plotly_white",
              title="Log-distance between Z and Z' with respect to ADMM convergence rates")
fig.update_traces(mode='markers+lines')
fig.show()

In [ ]:
time_dict = dict()
accuracy_dict = dict()
Z_dict = dict()

for X, S in zip(list(X_dict.values()), list(S_dict.values())):
    times, accs, precs = time_benchmark(X=X, S=S, Z_model="sklearn", lambda1=0.1, n_iter=5,
                                        sk_params=sk_params, rg_params=rg_params, admm_params=admm_params)
    
    time_dict.update(times)
    accuracy_dict.update(accs)
    Z_dict.update(precs)

In [ ]:
sparsity = sparsity_benchmark(Theta_dict=Theta_dict, Z_dict=Z_dict, t_rounding=1e-4)

In [ ]:
df = dict_to_dataframe(times=time_dict, acc_dict=accuracy_dict, spars_dict=sparsity)
df.head()

In [ ]:
fig = plot_log_distance(df, upper_bound=0.01, lower_bound=0.001)
fig.show()

In [ ]:
b = df[(df["accuracy"] < 0.01) & (df["accuracy"] > 0.001)]

In [ ]:
b.dtypes

In [ ]:
names = dict()
frames = dict()
for p in b["p"].unique():
    for method in b["method"].unique():
        names[method] = b[(b["p"] == p) & (b["method"] == method)]["hamming"].min()
    frame = pd.DataFrame(names.items(), columns=['method', 'min_hamming'])
    frame = frame.sort_values(by='min_hamming', ascending=True)
    frames[p] = frame.reset_index(drop=True)

In [ ]:
frames[100]

In [ ]:
frames[300]

In [ ]:
frames[1000]